(accessing_packages)=

# Accessing packages

In [1]:
from enviPath_python.enviPath import *
import getpass

As a first step to initialize the enviPath class, one needs to instantiate the *id*, in this case the host, from which data will be retrieved. This is set up in this way so that the user can decide which instance to use, i.e. prod, dev or for developers, their own local instance ([http://localhost:8080/](http://localhost:8080/)).

Each enviPath object also requires an *id*, as you might know from the documentation or other tutorials, ids are on enviPath URLs to the corresponding object page, in the example below, since we want the data contained on EAWAG-BBD package, we will therefore use the URL to [EAWAG-BBD](https://envipath.org/package/32de3cf4-e3e6-4168-956e-32fa5ddb0ce1)

In [8]:
HOST_INSTANCE = "https://envipath.org"
EAWAG_BBD_URL = "https://envipath.org/package/32de3cf4-e3e6-4168-956e-32fa5ddb0ce1"
eP = enviPath(HOST_INSTANCE)
pkg = Package(eP.requester, id=EAWAG_BBD_URL)
selected_pathways = pkg.get_pathways()[:2]
for pathway in selected_pathways:
    print(f"Pathway {pathway.get_name()} can be found on: {pathway.get_id()}")

Pathway (+)-Camphor can be found on: https://envipath.org/package/32de3cf4-e3e6-4168-956e-32fa5ddb0ce1/pathway/8ee9cf25-ece9-4c21-886c-23936be81f30
Pathway 1,1,1-Trichloro-2,2-bis-(4`-chlorophenyl)ethane (DDT) can be found on: https://envipath.org/package/32de3cf4-e3e6-4168-956e-32fa5ddb0ce1/pathway/3f58e4d4-1c63-4b30-bf31-7ae4b98899fe


This works well for packages that are public. However, when one wants to access private data the `login` method from `enviPath` class must be invoked. We use this opportunity to recommend the usage of getpass library in order to avoid password leaks when sharing code among pears.

In [3]:
username = "myusername"
password = "mypassword" # We recommend to use here: getpass.getpass()

PRIVATE_PACKAGE_URL = "https://envipath.org/package/8dc6c079-d43f-4eb7-afda-fcacb94699a5"
try:
    eP.login(username, password)
    private_pkg = Package(eP.requester, id=PRIVATE_PACKAGE_URL)
except Exception as e:
    print(f"An exception is raised due to the dummy credentials we used: {e}")

An exception is raised due to the dummy credentials we used: Login Failed!


Finally, we show that one can review the amount of existing data on the reviewed packages with just a few lines of code. In this tutorial, we use [Plotly](https://plotly.com/python/) to visualize the package statistics.

In [30]:
import pandas as pd
import plotly.graph_objects as go


# Get all reviewed packages
reviewed_packages = [
    Package(eP.requester, id=EAWAG_BBD_URL),
    Package(eP.requester, id="https://envipath.org/package/7932e576-03c7-4106-819d-fe80dc605b8a"), # EAWAG-SLUDGE
    Package(eP.requester, id="https://envipath.org/package/5882df9c-dae1-4d80-a40e-db4724271456") # EAWAG-SOIL
]

# Fetch all the relevant data
pkg_data = {}
for pkg in reviewed_packages:
    d = {}
    d["Compounds"] = len(pkg.get_compounds())
    d["Reactions"] = len(pkg.get_reactions())
    d["Rules"] = len(pkg.get_rules())
    d["Scenarios"] = len(pkg.get_scenarios())
    d["Pathways"] = len(pkg.get_pathways())
    pkg_data[pkg.get_name()] = d

data = pd.DataFrame(pkg_data)
fig = go.Figure()

# Generate a bar plot with a dropdown menu for a more comfortable selection and visualization of packages
button_list = [
    dict(
        label="All",
        method="update",
        args=[{"visible": [True]*len(data.columns)},
              {"title": "Package statistics of all reviewed packages"}])
]
for (i, package_name) in enumerate(data.columns):
    fig.add_trace(
        go.Bar(x=data.index, y=data[package_name], name=package_name, visible=i==0)
    )
    button = dict(
        label=package_name,
        method="update",
        args=[{"visible": [val==i for val in range(len(data.columns))]},
              {"title": f"Package statistics of {package_name}"}])
    button_list.append(button)
i = 0
fig.update_layout(
    title = f"Package statistics of {data.columns[i]}",
    updatemenus=[
        dict(
            active=i+1,
            buttons=list(button_list),
        )
    ])

fig
